In [ ]:
from endpoints import SofaScore
from datetime import datetime, timedelta
from helpers import send_request, generate_date_list, generate_live_games, fetch_event_data
import pandas as pd
import matplotlib.pyplot as plt


# Establish todays date
# start_date = datetime.today().strftime('%Y-%m-%d')

start_date = '2023-01-20'

# Establish an end date
end_date = '2023-01-01'

# Generate a list of dates
dates = generate_date_list(start_date,end_date)

# Create a list of endpoints from the list of dates and the Sofa base_endpoint
dated_endpoints = [SofaScore.BASE_ENDPOINT + date for date in dates]


In [ ]:
from helpers import send_request, generate_date_list, generate_live_games, fetch_event_data

# store list of responses from the urls
responses = [response for response in await fetch_event_data(dated_endpoints) if response['events']]




In [ ]:
def extract_event_data(response_array):

    # Initialize an empty list to store the event data
    event_data = []

    for response in response_array:
        # Filter out events that are not postponed

        events = [event for event in response['events'] if event['status']['type'] not in ['postponed', 'canceled']]
        
        # Loop through each event
        for event in events:   
                    
            # sometime score dicts have multiple scores, so just take max value; they should be same. 
            score_key = max(event['homeScore'], key = event['homeScore'].get) if event['homeScore'] else None
            
            # if no score obj, then skip
            if score_key:
                # Create a dictionary to store the relevant data for each event
                struct = {
                    'tournament' : event['tournament']['name'],  # Name of the tournament
                    'region' : event['tournament']['category']['name'],  # Region of the tournament
                    'home_team' : event['homeTeam']['name'],  # Name of the home team
                    'away_team' : event['awayTeam']['name'],  # Name of the away team
                    'home_score': event['homeScore'][score_key],  # Current score of the home team
                    'away_score': event['awayScore'][score_key],  # Current score of the away team
                    'total'     : event['homeScore'][score_key] + event['awayScore'][score_key],  # Total score
                    'start_timeestamp' : event['startTimestamp'],  # Start timestamp of the event
                    'game_id' : event['id'] # Unique Game ID
                    # 'start_datetime' : convert_timestamp_to_est_date(event['startTimestamp'])  # Start datetime of the event in EST
                }
                
                # Append the dictionary to the list of event data
                event_data.append(struct)
    
    # Return the list of event data
    return event_data

In [ ]:
event_data = extract_event_data(responses)

In [ ]:
# create endpoints to each specific game contained within event_data

from endpoints import SofaScore

def extract_specific_games(events):
    unique_id_urls = []
    for games in events:
        game_id_url = SofaScore.INDIVIDUAL_ENDPOINT.replace('UNIQUEGAMEID', str(games['game_id']))
        unique_id_urls.append(game_id_url)
    


    return unique_id_urls

unique_id_urls = extract_specific_games(event_data)

In [ ]:
# Fetch indivudal game data

game_responses = [gameRep for gameRep in await fetch_event_data(unique_id_urls[:50])]


In [ ]:
# Iterate over individual game data to gather dictionary for SofaScores expectations on the game

expectations_response = []

for response in game_responses:
    for stats in response['statistics']:
        for groups in stats['groups']:
            # type = groups['groupName']
            for expectations in groups['statisticsItems']:
                if expectations['name'] == 'Expected goals':
                    struct = {
                        'type_of_stat' : expectations['name'],  # Name of the tournament
                        'home_team_expectation' : expectations['home'],  # Name of the home team
                        'away_team_expectation' : expectations['away'],  # Name of the away team
                    }
                    
                    expectations_response.append(struct)
                else:
                    continue


for x in expectations_response:
    print(x)
                


In [ ]:
def calculate_winrate(event_data, threshold, **kwargs):
    """
    Credit to: https://github.com/rushcapital
    This function calculates the winrate of the events based on a given threshold.
    Parameters:
    event_data (list): A list of dictionaries containing event data.
    threshold (float): The threshold value to calculate the winrate.
    **kwargs: Additional filters to apply on the data.

    Returns:
    float: The calculated winrate.
    """

    # Convert the event data into a pandas DataFrame
    df = pd.DataFrame(event_data).drop_duplicates().reset_index(drop=True)
    
    # If there are additional filters, create a query string
    query = ' & '.join([f"{k} == '{v}'" for k, v in kwargs.items()]) if kwargs else None
    
    # Create the under and over queries based on the threshold and additional filters
    # The under query selects events where the total score is less than the threshold
    under_query = f"{query} & total < {threshold}" if query else f'total < {threshold}'
    
    # The over query selects events where the total score is more than the threshold
    over_query = f"{query} & total > {threshold}" if query else f'total > {threshold}'
    
    # Apply the under and over queries to the DataFrame
    under = df.query(under_query)
    over = df.query(over_query)
    
    # Calculate the winrate as the ratio of under events to the total number of events
    # If there are no events, the winrate is 0
    winrate = round(under.shape[0] / (under.shape[0] + over.shape[0]),3) if (under.shape[0] + over.shape[0]) != 0 else 0
    
    struct = {
        'total_games' : under.shape[0] + over.shape[0],
        'under_wins' : under.shape[0],
        'under_winrate' : winrate
    }

    return struct


In [ ]:
winrate_england_struct = calculate_winrate(event_data, threshold=3.5, region='England')


print(winrate_england_struct)


In [ ]:
# winrate_struct = calculate_winrate(event_data, threshold=3.5)
import pandas as pd

winrate_england_struct = calculate_winrate(event_data, threshold=3.5, region='England')
winrate_spain_struct = calculate_winrate(event_data, threshold=3.5, region='Spain')
winrate_italy_struct = calculate_winrate(event_data, threshold=3.5, region='Italy')
winrate_belgium_struct = calculate_winrate(event_data, threshold=3.5, region='Belgium')
winrate_greece_struct = calculate_winrate(event_data, threshold=3.5, region='Greece')

Data = pd.DataFrame([winrate_england_struct, winrate_spain_struct, winrate_italy_struct, winrate_belgium_struct, winrate_greece_struct])

import matplotlib.pyplot as plt

regions = ['England', 'Spain', 'Italy', 'Belgium', 'Greece']
winrates = Data['under_winrate']
x = range(len(regions))
plt.bar(regions, winrates)
plt.title('Under 3.5 Winrates by Region')
plt.xlabel('Region')
plt.ylabel('winrate %')
plt.show()




In [ ]:


# function to extract all regions

def extract_regions(response_array):

    unique_regions = []

    # Iterate over the response_array. (This should be event_data)

    for data in response_array:
        region = data['region']

        # If the Region exists in the unique_regions list, skip over it, else, append it.
        if region in unique_regions:
            continue
        else:
            unique_regions.append(region)

    # Return the list.
    return unique_regions

# Call to store the unique list of regions
list_of_regions = extract_regions(event_data)


